In [1]:
import time
import random
from math import ceil
from decimal import Decimal
from typing import List, Tuple

import random
from sympy import Symbol, Poly


# Use a large prime as the modulus for arithmetic operations
prime: int = 15485867
threshold: int = 5
secret1: int = 1234
secret2: int = 4321

In [2]:
# Client-side
# All parties use the same prime and same threshold
def sample_shamir_polynomial(zero_value: int) -> List[int]:
    coefs = [zero_value] + [random.randrange(prime) for _ in range(threshold - 1)]
    return coefs

In [3]:
# Client-side
def evaluate_at_point(coefs: List[int], point: int) -> int:
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % prime
    return result

In [4]:
# Server-side
def interpolate_at_point(points_values: List[Tuple[int, int]], point: int) -> int:
    points, values = zip(*points_values)
    constants = lagrange_constants_for_point(points, point)
    return sum(ci * vi for ci, vi in zip(constants, values)) % prime

In [5]:
# Server-side
def lagrange_constants_for_point(points: List[int], point: int) -> List[int]:
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - point)) % prime
                denum = (denum * (xj - xi)) % prime
        constants[i] = (num * pow(denum, -1, prime)) % prime
    return constants

In [6]:
# Client-side
def shamir_share(secret: int, num_shares: int) -> List[Tuple[int, int]]:
    polynomial = sample_shamir_polynomial(secret)
    print(polynomial)
    shares = [(i, evaluate_at_point(polynomial, i)) for i in range(1, num_shares + 1)]
    return shares

In [7]:
shares1 = shamir_share(secret1, 10)
shares2 = shamir_share(secret2, 10)

[1234, 8330410, 3058903, 4211912, 1388811]
[4321, 5984936, 262622, 11344372, 7424688]


In [8]:
# Server-side
def shamir_add(x, y):
    return [ (i+1, (xi[1] + yi[1]) % prime) for i, (xi, yi) in enumerate(list(zip(x, y))) ]

In [9]:
added = shamir_add(shares1, shares2)

In [10]:
# Server-side
def shamir_reconstruct(shares: List[Tuple[int, int]]) -> int:
    polynomial = [(p, v) for p, v in shares]
    secret = interpolate_at_point(polynomial, 0)
    return secret

In [11]:
shamir_reconstruct(added)

5555